In [1]:
# отключим всякие предупреждения Anaconda
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
from sklearn.metrics.regression import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.linear_model import LinearRegression, LassoCV, Lasso
from sklearn.ensemble import RandomForestRegressor

In [2]:
data = pd.read_csv('../../Desktop/winequality-white.csv', sep=';')

In [3]:
data.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4898 entries, 0 to 4897
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         4898 non-null   float64
 1   volatile acidity      4898 non-null   float64
 2   citric acid           4898 non-null   float64
 3   residual sugar        4898 non-null   float64
 4   chlorides             4898 non-null   float64
 5   free sulfur dioxide   4898 non-null   float64
 6   total sulfur dioxide  4898 non-null   float64
 7   density               4898 non-null   float64
 8   pH                    4898 non-null   float64
 9   sulphates             4898 non-null   float64
 10  alcohol               4898 non-null   float64
 11  quality               4898 non-null   int64  
dtypes: float64(11), int64(1)
memory usage: 459.3 KB


### Отделите целевой признак, разделите обучающую выборку в отношении 7:3 (30% - под оставленную выборку, пусть random_state=17) и отмасштабируйте данные с помощью StandardScaler.

In [35]:
X, y = data.drop('quality', axis=1), data['quality']

X_train, X_holdout, y_train, y_holdout = train_test_split(X, y, 
                                                          test_size=0.3, 
                                                          random_state=17)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_holdout_scaled = scaler.transform(X_holdout)

In [6]:
linreg = LinearRegression()
linreg.fit(X_train_scaled, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

## Вопрос 1: Каковы среднеквадратичные ошибки линейной регрессии на обучающей и отложенной выборках?

In [7]:
print("Mean squared error (train): %.3f" % mean_squared_error(y_train, linreg.predict(X_train_scaled)))
print("Mean squared error (test): %.3f" % mean_squared_error(y_holdout, linreg.predict(X_holdout_scaled)))

Mean squared error (train): 0.558
Mean squared error (test): 0.584


#### Посмотрите на коэффициенты модели и отранжируйте признаки по влиянию на качество вина (учтите, что большие по модулю отрицательные значения коэффициентов тоже говорят о сильном влиянии). Создайте для этого новый небольшой DataFrame.
Вопрос 2: Какой признак линейная регрессия считает наиболее сильно влияющим на качество вина: residual sugar

In [32]:
linreg_coef = pd.DataFrame(linreg.coef_, 
             data.columns[:-1], columns=['coef'])
linreg_coef.sort_values(by='coef',ascending=False)

,coef
residual sugar,0.538164
pH,0.150036
alcohol,0.129533
fixed acidity,0.097822
sulphates,0.062053
free sulfur dioxide,0.042180
total sulfur dioxide,0.014304
chlorides,0.008127
citric acid,-0.000183
volatile acidity,-0.192260


### Обучите Lasso-регрессию с небольшим коэффициентом α=0.01 (слабая регуляризация). Пусть опять random_state=17.

In [9]:
lasso1 = Lasso(alpha=0.01, random_state=17)
lasso1.fit(X_train_scaled, y_train)

Lasso(alpha=0.01, copy_X=True, fit_intercept=True, max_iter=1000,
      normalize=False, positive=False, precompute=False, random_state=17,
      selection='cyclic', tol=0.0001, warm_start=False)

#### Посмотрите на коэффициенты модели и отранжируйте признаки по влиянию на качество вина. Какой признак "отвалился" первым, то есть наименее важен для объяснения целевого признака в модели Lasso?

## fixed acidity

In [10]:
lasso1_coef = pd.DataFrame(lasso1.coef_, 
             data.columns[:-1], columns=['coef'])
lasso1_coef.sort_values(by='coef',ascending=False)

,coef
alcohol,0.322425
residual sugar,0.256363
pH,0.067277
free sulfur dioxide,0.043088
sulphates,0.029722
fixed acidity,-0.000000
citric acid,-0.000000
total sulfur dioxide,-0.000000
chlorides,-0.002747
volatile acidity,-0.188479


### Теперь определите лучшее значение α в процессе кросс-валидации 5-кратной кросс-валидации. Используйте LassoCV и random_state=17.

In [11]:
alphas = np.logspace(-6, 2, 200)
lasso_cv = LassoCV(alphas=alphas, cv=5, random_state=17)
lasso_cv.fit(X_train_scaled, y_train)

LassoCV(alphas=array([1.00000000e-06, 1.09698580e-06, 1.20337784e-06, 1.32008840e-06,
       1.44811823e-06, 1.58856513e-06, 1.74263339e-06, 1.91164408e-06,
       2.09704640e-06, 2.30043012e-06, 2.52353917e-06, 2.76828663e-06,
       3.03677112e-06, 3.33129479e-06, 3.65438307e-06, 4.00880633e-06,
       4.39760361e-06, 4.82410870e-06, 5.29197874e-06, 5.80522552e-06,
       6.36824994e-06, 6.98587975e-0...
       3.61234270e+01, 3.96268864e+01, 4.34701316e+01, 4.76861170e+01,
       5.23109931e+01, 5.73844165e+01, 6.29498899e+01, 6.90551352e+01,
       7.57525026e+01, 8.30994195e+01, 9.11588830e+01, 1.00000000e+02]),
        copy_X=True, cv=5, eps=0.001, fit_intercept=True, max_iter=1000,
        n_alphas=100, n_jobs=None, normalize=False, positive=False,
        precompute='auto', random_state=17, selection='cyclic', tol=0.0001,
        verbose=False)

In [12]:
lasso_cv.alpha_

0.0002833096101839324

### Выведите коэффициенты "лучшего" Lasso в порядке убывания влияния на качество вина.
Вопрос 3: Какой признак "обнулился первым" в настроенной модели LASSO?

# citric acid

In [12]:
lasso_cv_coef = pd.DataFrame(lasso_cv.coef_, 
             data.columns[:-1], columns=['coef'])
lasso_cv_coef.sort_values(by='coef',ascending=False)

,coef
residual sugar,0.526883
pH,0.146549
alcohol,0.137115
fixed acidity,0.093295
sulphates,0.060939
free sulfur dioxide,0.042698
total sulfur dioxide,0.012969
chlorides,0.006933
citric acid,-0.000000
volatile acidity,-0.192049


### Оцените среднеквадратичную ошибку модели на обучающей и тестовой выборках.
Вопрос 4: Каковы среднеквадратичные ошибки настроенной LASSO-регрессии на обучающей и отложенной выборках?

In [13]:
print("Mean squared error (train): %.3f" % mean_squared_error(y_train, lasso_cv.predict(X_train_scaled)))
print("Mean squared error (test): %.3f" % mean_squared_error(y_holdout, lasso_cv.predict(X_holdout_scaled)))

Mean squared error (train): 0.558
Mean squared error (test): 0.583


In [33]:
forest = RandomForestRegressor(random_state=17)
forest.fit(X_train_scaled, y_train)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=None, oob_score=False,
                      random_state=17, verbose=0, warm_start=False)

In [26]:
cross_val_score?

In [38]:
print("Mean squared error (train): %.3f" % mean_squared_error(y_train, forest.predict(X_train_scaled)))
print("Mean squared error (cv): %.3f" % np.mean(np.abs(cross_val_score(forest, X_train_scaled, y_train,
                                                                       scoring='neg_mean_squared_error'))))
print("Mean squared error (test): %.3f" % mean_squared_error(y_holdout, forest.predict(X_holdout_scaled)))

Mean squared error (train): 0.053
Mean squared error (cv): 0.414
Mean squared error (test): 0.372


### Настройте параметры min_samples_leaf и max_depth с помощью GridSearchCV и опять проверьте качество модели на кросс-валидации и на отложенной выборке.

In [101]:
forest_params = {'max_depth': list(range(10, 25)), 
                 'min_samples_leaf': list(range(1, 8)),
                 'max_features': list(range(6,12))}

locally_best_forest = GridSearchCV(forest, forest_params, n_jobs=-1)
locally_best_forest.fit(X_train_scaled, y_train)

GridSearchCV(cv=None, error_score=nan,
             estimator=RandomForestRegressor(bootstrap=True, ccp_alpha=0.0,
                                             criterion='mse', max_depth=None,
                                             max_features='auto',
                                             max_leaf_nodes=None,
                                             max_samples=None,
                                             min_impurity_decrease=0.0,
                                             min_impurity_split=None,
                                             min_samples_leaf=1,
                                             min_samples_split=2,
                                             min_weight_fraction_leaf=0.0,
                                             n_estimators=100, n_jobs=None,
                                             oob_score=False, random_state=17,
                                             verbose=0, warm_start=False),
             iid='deprecated', n_j

In [102]:
locally_best_forest.best_params_, locally_best_forest.best_score_

({'max_depth': 21, 'max_features': 6, 'min_samples_leaf': 1},
 0.4939384585901731)

### К сожалению, результаты GridSearchCV не полностью воспроизводимы (могут отличаться на разных платформах даже при фиксировании random_state). Поэтому обучите лес с параметрами max_depth=19, max_features=7, и min_samples_leaf=1 (лучшие в моем случае).


In [49]:
forest2 = RandomForestRegressor(max_depth=19, max_features=7, min_samples_leaf=1, random_state=17)
forest2.fit(X_train_scaled, y_train)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=19, max_features=7, max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=None, oob_score=False,
                      random_state=17, verbose=0, warm_start=False)

### Вопрос 6: Каковы среднеквадратичные ошибки настроенного случайного леса на обучающей выборке, на кросс-валидации (cross_val_score с scoring='neg_mean_squared_error') и на отложенной выборке?

In [96]:
print("Mean squared error (train): %.3f" % mean_squared_error(y_train, forest2.predict(X_train_scaled)))
print("Mean squared error (cv): %.3f" % cross_val_score(forest2, X_train_scaled, y_train, scoring='neg_mean_squared_error').mean())
print("Mean squared error (test): %.3f" % mean_squared_error(y_holdout, forest2.predict(X_holdout_scaled)))

Mean squared error (train): 0.058
Mean squared error (cv): -0.404
Mean squared error (test): 0.370


### Оцените важность признаков с помощью случайного леса.
Вопрос 7: Какой признак оказался главным в настроенной модели случайного леса?

## alcohol

In [62]:
rf_importance = pd.DataFrame(forest2.feature_importances_,
                      data.columns[:-1], columns=['importances'])
rf_importance.sort_values(by='importances',ascending=False)

,importances
alcohol,0.214825
volatile acidity,0.117627
free sulfur dioxide,0.113509
density,0.086530
pH,0.072646
residual sugar,0.072529
total sulfur dioxide,0.071644
chlorides,0.067819
fixed acidity,0.061873
citric acid,0.061500


In [111]:
mean_squared_error(y_holdout, forest.predict(X_holdout_scaled))

0.37163775510204083